In [19]:
import re
import nltk
import json
import gzip
import pandas as pd
porter = nltk.PorterStemmer()
del_non_letters = re.compile(ur'[^a-z ]')

In [3]:
del_non_letters.sub('', "safvsv 'dsv")

'safvsv dsv'

In [2]:
persons = list(l.strip().decode('utf-8') for l in open('persons.txt'))

In [6]:
persons[:3]

[u'doug band', u'tim kaine', u'rick']

In [3]:
# porter.stem()
def norm_name(name):
    return frozenset(sorted(del_non_letters.sub(u'', porter.stem(w)) for w in nltk.word_tokenize(name)))
persons_norm = list(norm_name(n) for n in persons)
persons_norm[:3]

[frozenset({u'band', u'doug'}),
 frozenset({u'kain', u'tim'}),
 frozenset({u'rick'})]

In [4]:
srt_persons_norm = sorted(persons_norm, key=lambda x : len(x), reverse=True)
srt_persons_norm[:3]

[frozenset({u'f', u'john', u'kennedi'}),
 frozenset({u'joseph', u'paul', u'watson'}),
 frozenset({u'craig', u'paul', u'robert'})]

In [5]:

unique_names = set()

for i, p in enumerate(srt_persons_norm):
    repeated = False
    for s in unique_names:
        if s > p:
            repeated = True
            break
    if not repeated:
        unique_names.add(p)
len(unique_names), len(srt_persons_norm)

(212, 318)

In [11]:
print "\n".join(sorted(" ".join(n) for n in unique_names))

abedin huma
adolf hitler
al gore
alex jone
allen
ami goodman
ami moreno
ammon bundi
anthoni weiner
aoun
arnaldo rodger
ash carter
assad bashar
austin fitt
band doug
barack obama
bashar alassad
benjamin netanyahu
berni sander
bernish clair
beyonc
biden joe
bill clinton
billi okeef
black
bob
brazil donna
breitbart
bret baier
brexit
brown
burton
bush w georg
carney jay
chaffetz jason
charl
chavez hugo
cheryl mill
chomski
christ jesu
christi
clapper jame
clinton chelsea
clinton hilari
cohen
cooper
craig paul robert
cruz ted
daesh
daniel
davi
dick cheney
donald j trump
doss
duke david
dunford
durden tyler
dylan
eddi
eichenwald kurt
eisenhow
elizabeth warren
ellison
eowyn
erdogan
eva
evan mcmullin
farag nigel
ferguson
flower
flynn
fox
franci pope
fukushima
gaddafi muammar
geoffrey grider
giuliani rudi
glen ford
gruber
hamilton
hanniti sean
hayden
hebdo charli
hill
hillaryclinton
hodg dave
holder eric
holland
hw georg
ick
ivanka trump
j arnoldski
jackson
jame comey
jason ditz
jeffrey epstein


In [6]:
from collections import defaultdict
f = gzip.open('kaggle_f_n_t_sentences.txt.gz')

uuid = None
persons_dict = defaultdict(float)
uuid_features = []

all_persons_words = set()
for p in unique_names:
    all_persons_words.update(p)

unique_names_cache = {}
def get_unique_name(name):
    if name in unique_names_cache:
        return unique_names_cache.get(name)
    n_name = norm_name(name)
    for w in n_name:
        if w not in all_persons_words:
            return None
    for s in unique_names:
        if s >= n_name:
            unique_names_cache[name] = s
            return s
    return None



In [17]:
trans_dict = {}
f = gzip.open('kaggle_trans_feat.txt.gz')
for i, l in enumerate(f):
    uuid, feat = l.strip('\n').split('\t')
    trans_dict[uuid] = json.loads(feat)
    

In [28]:
tbl2 = pd.read_csv('data_sample.csv')
web_features = {}
ground_true = {}
tbl2 = tbl2.fillna(0)
# domain_rank, spam_score, replies_count, participants_count, likes, comments, shares
for row in tbl2.iterrows():
    if row[1]['language'] != 'english':
        continue
    ground_true[row[1]['uuid']] = 1 if row[1]['type'] == 'ok' else 0
    web_features[row[1]['uuid']] = [row[1]['domain_rank'], row[1]['spam_score'],
                                   row[1]['replies_count'],row[1]['participants_count'],
                                   row[1]['likes'],row[1]['comments'],
                                   row[1]['shares']]
columns = ['domain_rank', 'spam_score', 'replies_count', 'participants_count', 'likes', 'comments', 'shares']

In [29]:
names = sorted(" ".join(n) for n in unique_names)
def make_features(persons):
    p2 = dict((k, v+0.5 if v>=0 else v-0.5) for k, v in persons.iteritems())
    return list((p.replace(' ', '_'), p2.get(p, 0)) for p in names)

f = open('uuid_feat.txt')
f_res = open('final_feat.txt', 'w')
f_res.write("uuid\tground_true\t{0}\tpos\tneg\t{1}\t{2}\n".format(
                    "\t".join(columns),
                    "\t".join(p.replace(' ', '_') for p in names),
                    '\t'.join("t{0}".format(i) for i in range(500))))
for i, l in enumerate(f):
    l = l.strip('\n').split('\t')
    if l[0] not in ground_true:
        continue
    feat = make_features(json.loads(l[3]))
    f_res.write("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\n".format(l[0],
                                                             ground_true[l[0]],
                                                             '\t'.join(map(str, web_features[l[0]])),
                                                             l[1], l[2], '\t'.join(str(t[1]) for t in feat), '\t'.join(map(str, trans_dict[l[0]]))))
#     print l[0], l[1], l[2], l[3]
#     if i > 20:
#         break
